In [ ]:
import numpy as np
import pydicom
import pydicom_seg
import nibabel as nib
dcm = pydicom.dcmread('./SEG_20240507_121615_030_S6.dcm')


reader = pydicom_seg.MultiClassReader()
result = reader.read(dcm)

image_data = result.data  # directly available
image = result.dataset  # lazy construction


total_slices = len(image.ReferencedSeriesSequence[0].ReferencedInstanceSequence)
slice_positions = [x.FrameContentSequence[0].DimensionIndexValues[1] for x in image.PerFrameFunctionalGroupsSequence]

nii_arr = np.zeros((image_data.shape[1], image_data.shape[2], total_slices))
for i, slice in enumerate(image_data):
    nii_arr[:, :, slice_positions[i]] = slice

nib.save(nib.Nifti1Image(nii_arr, np.eye(4)), 'seg.nii')

2024-10-08 11:00:51,869 [WARNING] DICOM-SEG does not specify "(0062, 0013) SegmentsOverlap", assuming UNDEFINED and checking pixels


In [11]:
nib.load('input_Ax_T2_FLAIR_20120920000000_6.nii').header

In [12]:
slice_positions = [x.FrameContentSequence[0].DimensionIndexValues[1] for x in image.PerFrameFunctionalGroupsSequence]
slice_positions

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [1]:
import pydicom
dcm = pydicom.dcmread('./SEG_20240507_121548_916_S9.dcm')
dcm

FileNotFoundError: [Errno 2] No such file or directory: './SEG_20240507_121548_916_S9.dcm'

In [1]:
from dcmseg2nii import main
print(main('in/SEG_20240507_121615_030_S6.dcm', 'AXIAL_T2_FLAIR.nii.gz', 'in/'))
print(main('in/SEG_20240507_121548_916_S9.dcm', 'Ax_T1_SE_+C.nii.gz', 'in/'))

Number of segments: 1
['in/AXIAL_T2_FLAIR_Edema.nii.gz']
Number of segments: 2
['in/Ax_T1_SE_+C_Mass.nii.gz', 'in/Ax_T1_SE_+C_Necrosis.nii.gz']


In [2]:
import nibabel as nib
import SimpleITK as sitk
image = nib.load('Ax_T1_SE_+C.nii.gz')
image = sitk.ReadImage('Ax_T1_SE_+C.nii.gz')
import numpy as np
def get_affine_from_sitk(image) -> np.ndarray:
    """Extract affine matrix from SimpleITK image."""
    
    # Get components
    direction = image.GetDirection()
    origin = image.GetOrigin()
    spacing = image.GetSpacing()
    
    # Reshape direction to 3x3 matrix
    direction_matrix = np.array(direction).reshape(3,3)
    
    # Create affine matrix
    affine = np.eye(4)
    
    # Fill rotation and spacing
    for i in range(3):
        for j in range(3):
            affine[i,j] = direction_matrix[i,j] * spacing[j]
    
    # Fill translation
    affine[:3,3] = origin
    
    return affine
nib.aff2axcodes(get_affine_from_sitk(image))

('R', 'P', 'S')

In [ ]:
# 1. Import necessary libraries
import tempfile
import xnat
import os
import logging
from dcmseg2nii import main

import pydicom
from datetime import datetime, timedelta

# 2. Configure logging to both file and console
logging.basicConfig(
    level=logging.INFO,  # Set to INFO or DEBUG for more detailed logs
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("dcmseg2nii_conversion.log"),
        logging.StreamHandler()
    ]
)

def extract_code_meanings(dicomseg_path: str) -> list[str]:
    """
    Extracts the segmented code meanings from a DICOM SEG file.

    Parameters:
        dicomseg_path (str): Path to the DICOM SEG file.

    Returns:
        List[str]: A list of code meanings.
    """
    logging.info("Extracting code meanings from DICOM SEG...")
    dcm = pydicom.dcmread(dicomseg_path)
    code_meanings = []
    if 'SegmentSequence' in dcm:
        for segment in dcm.SegmentSequence:
            # Extract the Segmented Property Type Code Meaning
            if ('SegmentedPropertyTypeCodeSequence' in segment and
                len(segment.SegmentedPropertyTypeCodeSequence) > 0):
                type_code = segment.SegmentedPropertyTypeCodeSequence[0]
                if 'CodeMeaning' in type_code:
                    code_meanings.append(type_code.CodeMeaning)
    logging.info("Extracted code meanings: %s", code_meanings)
    return code_meanings

def extract_series_uid(dicomseg_path: str) -> str:
    """
    Extracts the Series Instance UID from a DICOM SEG file.

    Parameters:
        dicomseg_path (str): Path to the DICOM SEG file.

    Returns:
        str: Series Instance UID.
    """
    logging.info(f"Extracting Series Instance UID from {dicomseg_path}...")
    try:
        dcm = pydicom.dcmread(dicomseg_path, stop_before_pixels=True)
        series_uid = dcm.ReferencedSeriesSequence[0].SeriesInstanceUID
        logging.info(f"Extracted Series Instance UID: {series_uid}")
        return series_uid
    except Exception as e:
        logging.error(f"Failed to extract Series Instance UID from {dicomseg_path}: {e}")
        raise


def get_series_name_from_xnat(session, series_uid: str, experiment) -> str:
    """
    Retrieves the series name from XNAT by iterating through all scans to find a match for the Series Instance UID.

    Parameters:
        session: XNATpy session object.
        series_uid (str): Series Instance UID.
        project_id (str): XNAT project ID.

    Returns:
        str: Series name or 'UnknownSeries' if not found.
    """
    logging.info(f"Searching for Series Instance UID: {series_uid} in project: {project_id}...")
    try:
        for scan in experiment.scans.values():
            scan_series_uid = scan.uid
            # print(scan_series_uid)
            if scan_series_uid == series_uid:
                series_name = scan.series_description
                series_name_clean = series_name.replace(' ', '_').replace('/', '_')
                logging.info(f"Found Series Name: {series_name_clean} for Series UID: {series_uid}")
                return series_name_clean
        logging.warning(f"No scan found with Series Instance UID: {series_uid} in Experiment: {experiment.label}")
        return 'UnknownSeries'
    except Exception as e:
        logging.error(f"Error while searching for Series UID {series_uid}: {e}")
        return 'UnknownSeries'

def get_series_name_from_xnat(session, series_uid: str, experiment) -> str:
    """
    Retrieves the series name from XNAT by iterating through all scans to find a match for the Series Instance UID.

    Parameters:
        session: XNATpy session object.
        series_uid (str): Series Instance UID.
        experiment: The experiment object from XNAT.

    Returns:
        str: Series name or 'UnknownSeries' if not found.
    """
    logging.info(f"Searching for Series Instance UID: {series_uid} in experiment: {experiment.label}...")
    try:
        for scan in experiment.scans.values():
            if scan.uid == series_uid:
                series_name = scan.series_description
                series_name_clean = series_name.replace(' ', '_').replace('/', '_')
                logging.info(f"Found Series Name: {series_name_clean} for Series UID: {series_uid}")
                return series_name_clean
        logging.warning(f"No scan found with Series Instance UID: {series_uid} in experiment: {experiment.label}")
        return 'UnknownSeries'
    except Exception as e:
        logging.error(f"Error while searching for Series UID {series_uid}: {e}")
        return 'UnknownSeries'


# 3. Define XNAT connection parameters using environment variables for security
xnat_url = os.getenv('XNAT_URL') 
username = os.getenv('XNAT_USERNAME')
password = os.getenv('XNAT_PASSWORD')

# 4. Define the project ID and output directory
project_id = 'BRAIN'                   # Replace with your project ID

data_prefix = '/Volumes/Siren/Brain_data/1.PatientDirectory'
session_prefix = "MRI/Processed_New/"

# 5. Connect to XNAT and process segmentations
# WARNING: Ensure XNAT_URL, XNAT_USERNAME, and XNAT_PASSWORD environment variables are set
with xnat.connect(xnat_url, user=username, password=password) as session:
    project = session.projects[project_id]
    
    for subject in project.subjects.values():
        print(f"Subject: {subject.label}")
        
        # Example: Filter for a specific subject if needed
        # if subject.label != '150':
        #     continue  # Skip subjects that are not '150'
        
        for experiment in subject.experiments.values():
            print(f"  Experiment: {experiment.label}")
            
            for scan in experiment.assessors.values():
                for resource in scan.resources.values():
                    if resource.label == 'SEG':  # Adjust if your resource label differs
                        print(f"    Found DICOM SEG Resource in Scan {scan.label}")
                        
                        for dicom_file in resource.files.values():
                            print(f"      Processing {dicom_file.name}...")
                            
                            with tempfile.NamedTemporaryFile(suffix='.dcm', delete=False) as temp_dicom_file:
                                dicom_file.download(temp_dicom_file.name)
                                dicomseg_file_path = temp_dicom_file.name
                                print(f"        Downloaded {dicom_file.name} to temporary file.")
                            
                            try:
                                series_uid = extract_series_uid(dicomseg_file_path)
                            except Exception as e:
                                print(f"        Failed to extract Series UID from {dicom_file.name}: {e}")
                                os.remove(dicomseg_file_path)
                                print(f"        Removed temporary file {dicomseg_file_path}")
                                continue  # Skip to the next file
                            
                            try:
                                series_name_clean = get_series_name_from_xnat(session, series_uid, experiment)
                            except Exception as e:
                                print(f"        Failed to retrieve Series Name from XNAT for {dicom_file.name}: {e}")
                                series_name_clean = 'UnknownSeries'
                            
                            # convert date from offset
                            experiment_label_clean = experiment.label.replace(' ', '_').replace('/', '_')
                            experiment_date = experiment_label_clean.split('_')[1]
                            year = int(experiment_date[:4])
                            month = int(experiment_date[4:6])
                            day = int(experiment_date[6:])

                            # Subtract 8 days
                            date = datetime(year, month, day) - timedelta(days=8)

                            # Reconstruct the date string
                            new_experiment_date = f"{date.year:04d}{date.month:02d}{date.day:02d}"

                            # find the session directory, the directory has text beyond the date, so we need to find the match
                            directory_prefix = os.path.join(data_prefix, subject.label, session_prefix)
                            print(directory_prefix)
                            nii_directory = None
                            for directory in os.listdir(directory_prefix):
                                if new_experiment_date in directory:
                                    nii_directory = os.path.join(directory_prefix, directory)
                                    break
                            else:
                                print(f"        Could not find directory for {dicom_file.name}")
                                continue

                            # find the nii file in the directory
                            nii_path = None
                            for file in os.listdir(nii_directory):
                                if series_name_clean in file and '.nii' in file:
                                    nii_path = os.path.join(nii_directory, file)
                                    break
                            else:
                                if os.path.exists(os.path.join(nii_directory, 'T2.nii.gz')) and 'T2' in series_name_clean:
                                    nii_path = os.path.join(nii_directory, 'T2.nii.gz')
                                elif os.path.exists(os.path.join(nii_directory, 'FLAIR.nii.gz')) and 'FLAIR' in series_name_clean:
                                    nii_path = os.path.join(nii_directory, 'FLAIR.nii.gz')
                                elif os.path.exists(os.path.join(nii_directory, 'T1.nii.gz')) and 'T1' in series_name_clean:
                                    nii_path = os.path.join(nii_directory, 'T1.nii.gz')
                                elif os.path.exists(os.path.join(nii_directory, 'T1+C')) and 'T1+C' in series_name_clean:
                                    nii_path = os.path.join(nii_directory, 'T1C.nii.gz')
                                else:
                                    print(f"        Could not find NIfTI file for {dicom_file.name}")
                                continue

                            output_directory = os.path.join(nii_directory, 'xnat')
                            if not os.path.exists(output_directory):
                                os.makedirs(output_directory)

                            
                            # Convert DICOM SEG to aligned NIfTI
                            try:
                                main(dicomseg_file_path, nii_path, output_directory)
                            except Exception as e:
                                print(f"        Conversion failed for {dicom_file.name}: {e}")
                            
                            try:
                                os.remove(dicomseg_file_path)
                                print(f"        Removed temporary file {dicomseg_file_path}")
                            except Exception as e:
                                print(f"        Failed to remove temporary file {dicomseg_file_path}: {e}")

[WARNING] Cannot find matching class for fs:asegRegionAnalysis
2025-02-07 12:40:21,665 [WARNING] Cannot find matching class for fs:asegRegionAnalysis
[WARNING] Cannot find matching class for fs:automaticSegmentationData
2025-02-07 12:40:21,874 [WARNING] Cannot find matching class for fs:automaticSegmentationData
[WARNING] Cannot find matching class for fs:aparcRegionAnalysis
2025-02-07 12:40:21,875 [WARNING] Cannot find matching class for fs:aparcRegionAnalysis
[WARNING] Cannot find matching class for fs:longFSData
2025-02-07 12:40:21,876 [WARNING] Cannot find matching class for fs:longFSData
[WARNING] Cannot find matching class for fs:fsData
2025-02-07 12:40:22,017 [WARNING] Cannot find matching class for fs:fsData


Subject: 101
  Experiment: 101_20100827_mr
  Experiment: 101_20101020_mr
  Experiment: 101_20080705_ct
  Experiment: 101_20081218_mr
  Experiment: 101_20091111_mr
  Experiment: 101_20100129_mr
  Experiment: 101_20090508_mr
  Experiment: 101_20090904_mr
  Experiment: 101_20101006_mr
  Experiment: 101_20080801_mr
  Experiment: 101_20100716_mr
  Experiment: 101_20090401_mr
  Experiment: 101_20100507_mr
  Experiment: 101_20100604_mr
  Experiment: 101_20081022_mr
  Experiment: 101_20100327_mr
  Experiment: 101_20080626_ct
  Experiment: 101_20090703_mr
  Experiment: 101_20080627_mr
  Experiment: 101_20080626_mr
  Experiment: 101_20090130_mr
Subject: 102
  Experiment: 102_20090430_mr
  Experiment: 102_20110225_mr
  Experiment: 102_20100505_mr
  Experiment: 102_20080822_mr
  Experiment: 102_20090218_mr
  Experiment: 102_20110224_ct
  Experiment: 102_20080816_mr
  Experiment: 102_20080620_mr
  Experiment: 102_20090701_mr
  Experiment: 102_20090826_mr
  Experiment: 102_20080619_mr
  Experiment: 

  0% of 265.2 KiB |                                  |   0.0 s/B ETA:  --:--:--
100% of 265.2 KiB |################################|   3.3 MiB/s ETA:  00:00:00
100% of 265.2 KiB |################################|   3.3 MiB/s Time:  0:00:00
2025-02-07 12:40:48,816 [INFO] Extracting Series Instance UID from /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/tmpkssfkshv.dcm...
2025-02-07 12:40:48,819 [INFO] Extracted Series Instance UID: 2.25.295010520663807987161795204319385482851
2025-02-07 12:40:48,820 [INFO] Searching for Series Instance UID: 2.25.295010520663807987161795204319385482851 in experiment: 107_20120920_mr...


        Downloaded SEG_20240507_121548_916_S9.dcm to temporary file.


2025-02-07 12:40:49,230 [INFO] Found Series Name: Ax_T1_SE_+C for Series UID: 2.25.295010520663807987161795204319385482851


...

        Downloaded SEG_20240413_204832_240_S14.dcm to temporary file.


2025-02-07 13:08:33,792 [INFO] Found Series Name: Ax_T1_Post for Series UID: 2.25.85982146854832019178060356424734213577


/Volumes/Siren/Brain_data/1.PatientDirectory/227/MRI/Processed_New/
Number of segments: 1
        Removed temporary file /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/tmp3vuve792.dcm
    Found DICOM SEG Resource in Scan SEG_20240413_221528_558_S10
      Processing SEG_20240413_221528_558_S10.dcm...


  0% of 167.5 KiB |                                  |   0.0 s/B ETA:  --:--:--
100% of 167.5 KiB |################################|   2.4 MiB/s ETA:  00:00:00
100% of 167.5 KiB |################################|   2.4 MiB/s Time:  0:00:00
2025-02-07 13:08:44,522 [INFO] Extracting Series Instance UID from /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/tmpb6xqe9wb.dcm...
2025-02-07 13:08:44,529 [INFO] Extracted Series Instance UID: 2.25.81909074005509130134791363576157863995
2025-02-07 13:08:44,530 [INFO] Searching for Series Instance UID: 2.25.81909074005509130134791363576157863995 in experiment: 227_20221008_mr...
2025-02-07 13:08:44,530 [INFO] Found Series Name: Ax_FLAIR_FS for Series UID: 2.25.81909074005509130134791363576157863995


        Downloaded SEG_20240413_221528_558_S10.dcm to temporary file.
/Volumes/Siren/Brain_data/1.PatientDirectory/227/MRI/Processed_New/
Number of segments: 1
        Conversion failed for SEG_20240413_221528_558_S10.dcm: Segmentation size (216, 256, 42) does not match NIfTI size (256, 256, 42)
        Removed temporary file /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/tmpb6xqe9wb.dcm
  Experiment: 227_20220620_mr
  Experiment: 227_20211102_mr
  Experiment: 227_20211019_ct
  Experiment: 227_20211018_ct
  Experiment: 227_20211018_mr
  Experiment: 227_20220120_mr
  Experiment: 227_20211019_mr
  Experiment: 227_20211020_mr


In [1]:
from dcmseg2nii import convert_seg_to_full_series
annot_dict = convert_seg_to_full_series('SEG_20240507_121615_030_S6.dcm')

INFO:dcmseg2nii:Reading DICOM SEG file: SEG_20240507_121615_030_S6.dcm
INFO:dcmseg2nii:Extracting metadata from DICOM SEG file
INFO:dcmseg2nii:Decoding pixel data
INFO:dcmseg2nii:Extracting image positions
INFO:dcmseg2nii:Generating DICOM datasets for each segment
INFO:dcmseg2nii:Processing Segment 1: Edema


In [2]:
from dcmseg2nii import save_dicom_datasets_to_temp_directories, run_dcm2niix_on_directories
# Step 1: Save DICOM datasets to their own temporary directories
segment_dirs = save_dicom_datasets_to_temp_directories(annot_dict)

# Step 2: Run dcm2niix on the directories and get the NIfTI file paths
nifti_files = run_dcm2niix_on_directories(segment_dirs)

INFO:dcmseg2nii:Created temporary directory for segment Edema: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_001.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_002.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_003.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_004.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_005.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_lwvx1ucp/slice_006.dcm
INFO:dcmseg2nii:Saved DICOM file: /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/dicom_segment_Edema_

Chris Rorden's dcm2niiX version v1.0.20240202  Clang16.0.6 ARM (64-bit MacOS)
Found 22 DICOM file(s)
::autoBids:GE usedSeqName:'' seqName:'' internalSeqName:'' seriesDesc:'' scanSeq:'' stepDesc:'' bidsData:'' bidsSuffix:'_acq-_run-1'
Convert 22 DICOM as /var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/nifti_segment_Edema_l_y8gth8/dicom_segment_Edema_lwvx1ucp_Edema_0_1 (512x512x11x2)
Conversion required 0.057405 seconds (0.056497 for core code).


In [3]:
print(nifti_files)

{'Edema': '/var/folders/rc/rclfvgm135q8nz2l8y8dq38h0000gq/T/nifti_segment_Edema_l_y8gth8/dicom_segment_Edema_lwvx1ucp_Edema_0_1.nii.gz'}


In [2]:
import nibabel as nib
annot = nib.load('in/in_FH_ET_20120920000000_99.nii')
img = nib.load()

In [5]:
#!/usr/bin/env python3

import sys
import numpy as np
import pydicom
import nibabel as nib

# Highdicom Segmentation class:
from highdicom.seg import segread


def main(dicomseg_path, output_basename):
    """
    Convert a DICOM Segmentation file to one or more NIfTI label maps.
    Each segment in the DICOM-SEG will be saved as a separate NIfTI file.
    
    :param dicomseg_path: Path to the input DICOM Segmentation file (dicomseg).
    :param output_basename: The base filename/path to use for output NIfTI files.
    """
    # Read the DICOM SEG
    ds = pydicom.dcmread(dicomseg_path)
    
    # Initialize a highdicom Segmentation object
    seg = segread(dicomseg_path)
    
    # Iterate over each segment number in the segmentation
    for i, segment_number in enumerate(seg.segment_numbers):
        # Extract the 3D (or 4D) bitmap for this segment
        #   This returns a NumPy array with shape [z, y, x] if volumetric
        #   or [time, z, y, x] if time dimension is included.
        arr = seg.segment_pixel_array(segment_number)
        
        # Here, we use an identity affine just for demonstration.
        # In a real scenario, you would reconstruct the correct orientation
        # using the DICOM references for the original image series.
        affine = np.eye(4)
        
        # Create a NIfTI image
        nifti_img = nib.Nifti1Image(arr.astype(np.uint8), affine)
        
        # Write out the file. We append the segment number to the basename:
        out_file = f"{output_basename}_segment_{segment_number}.nii.gz"
        nib.save(nifti_img, out_file)
        print(f"Saved segment {segment_number} to {out_file}")


if __name__ == "__main__":
    # if len(sys.argv) != 3:
    #     print(f"Usage: {sys.argv[0]} <dicomseg_path> <output_nifti_basename>")
    #     sys.exit(1)

    # dicomseg_path = sys.argv[1]
    # output_basename = sys.argv[2]
    dicomseg_path = './in/SEG_20240507_121615_030_S6.dcm'
    output_basename = './seg'
    main(dicomseg_path, output_basename)

AttributeError: 'Segmentation' object has no attribute 'segment_pixel_array'

In [ ]:
import numpy as np
import SimpleITK as sitk
from pydicom import dcmread
import os


def inspect_dicom_seg(input_path):
    """Debug function to inspect DICOM-SEG structure"""
    dcm = dcmread(input_path)
    print(f"Pixel Array Shape: {dcm.pixel_array.shape}")
    print(f"Number of frames: {len(dcm.PerFrameFunctionalGroupsSequence)}")
    print(f"Referenced Series length: {len(dcm.ReferencedSeriesSequence[0].ReferencedInstanceSequence)}")
    print("\nFirst few DimensionIndexValues:")
    for i, frame in enumerate(dcm.PerFrameFunctionalGroupsSequence[:5]):
        print(f"Frame {i}: {frame.FrameContentSequence[0].DimensionIndexValues}")

def convert_dicomseg_to_nifti(input_path, output_path):
    """Convert DICOM-SEG to NIfTI"""
    dcm = dcmread(input_path)
    
    # Get dimensions
    rows = dcm.Rows
    cols = dcm.Columns
    num_slices = len(dcm.ReferencedSeriesSequence[0].ReferencedInstanceSequence)
    
    # Create output volume
    volume = np.zeros((num_slices, rows, cols), dtype=np.uint8)
    
    # Extract pixel data
    seg_data = dcm.pixel_array
    
    # Map frames to correct slices
    for frame_idx, frame in enumerate(dcm.PerFrameFunctionalGroupsSequence):
        dim_idx = frame.FrameContentSequence[0].DimensionIndexValues
        slice_idx = dim_idx[1] - 1  # Second value is slice number (1-based)
        
        frame_data = seg_data[frame_idx] if len(seg_data.shape) == 3 else seg_data
        volume[slice_idx] = np.logical_or(volume[slice_idx], frame_data)
    
    # Convert to SimpleITK image and flip for RAS orientation
    volume = np.flip(np.flip(volume, axis=1), axis=2)
    sitk_image = sitk.GetImageFromArray(volume)
    
    # Set metadata
    spacing = list(dcm.PixelSpacing) if hasattr(dcm, 'PixelSpacing') else [1.0, 1.0]
    spacing.append(dcm.SpacingBetweenSlices if hasattr(dcm, 'SpacingBetweenSlices') else 1.0)
    sitk_image.SetSpacing(spacing)
    
    # Save as NIfTI
    sitk.WriteImage(sitk_image, output_path)
    return True

if __name__ == "__main__":
    # Example usage
    dicomseg_path = './in/SEG_20240507_121615_030_S6.dcm'
    nifti_path = './seg.nii'
    ref_nifti_path = '/Volumes/Siren/Brain_data/2.PatientDirectory/107/MRI/Processed_New/20120912_Mr_Brain_Wo+_W_Cont_Ss/Ax_T2_FLAIR.nii.gz'
    convert_dicomseg_to_nifti(dicomseg_path, nifti_path)

In [27]:
import pydicom
import numpy as np
import nibabel as nib

def dicomseg_to_nifti(seg_path, out_nifti_path):
    """
    Convert a DICOM-SEG file to a full 3D NIfTI volume with the same
    orientation as dcm2niix would produce, relying solely on the DICOM-SEG metadata.
    """
    ds = pydicom.dcmread(seg_path)

    # Basic check: ensure file is a SEG
    if ds.Modality != "SEG":
        raise ValueError(f"File {seg_path} is not a segmentation (SEG). Found Modality={ds.Modality}.")

    # ----------------------------------------------------------------
    # 1) Identify all referenced slices from ReferencedSeriesSequence
    # ----------------------------------------------------------------
    if not hasattr(ds, "ReferencedSeriesSequence") or len(ds.ReferencedSeriesSequence) == 0:
        raise ValueError("No ReferencedSeriesSequence found in the SEG - cannot determine original slices.")

    ref_series = ds.ReferencedSeriesSequence[0]
    if not hasattr(ref_series, "ReferencedInstanceSequence") or len(ref_series.ReferencedInstanceSequence) == 0:
        raise ValueError("No ReferencedInstanceSequence found in the SEG - cannot determine original slices.")

    # This is the full list of SOPInstanceUIDs for each slice in the original series, in acquisition order:
    referenced_sop_uids = [inst.ReferencedSOPInstanceUID for inst in ref_series.ReferencedInstanceSequence]
    num_slices = len(referenced_sop_uids)

    # ----------------------------------------------------------------
    # 2) Gather orientation + spacing from SharedFunctionalGroups
    # ----------------------------------------------------------------
    # Typically found in ds.SharedFunctionalGroupsSequence
    shared_fg = ds.SharedFunctionalGroupsSequence[0]

    # Pixel spacing: [row spacing, column spacing]
    pixel_measures = shared_fg.PixelMeasuresSequence[0]
    pixel_spacing = pixel_measures.PixelSpacing
    spacing_row = float(pixel_spacing[0])
    spacing_col = float(pixel_spacing[1])

    # Slice thickness (may be approximate or absent)
    slice_thickness = None
    if hasattr(pixel_measures, "SliceThickness"):
        slice_thickness = float(pixel_measures.SliceThickness)

    # Orientation: row_dir (first 3), col_dir (last 3)
    orientation = shared_fg.PlaneOrientationSequence[0].ImageOrientationPatient
    row_dir = np.array(orientation[0:3], dtype=float)
    col_dir = np.array(orientation[3:6], dtype=float)
    # Normal direction (slice direction) is cross(row_dir, col_dir)
    # We'll finalize slice direction after we identify the specific slice positions below.

    # ----------------------------------------------------------------
    # 3) Prepare the empty segmentation volume
    # ----------------------------------------------------------------
    rows = ds.Rows
    cols = ds.Columns
    seg_volume = np.zeros((rows, cols, num_slices), dtype=np.uint8)

    # ----------------------------------------------------------------
    # 4) Map each Per-Frame functional group to the correct slice index
    # ----------------------------------------------------------------
    # ds.PerFrameFunctionalGroupsSequence[x].DerivationImageSequence or .ReferencedImageSequence
    # often includes the ReferencedSOPInstanceUID for that frame.
    # Also, we can read the plane position for computing an affine.
    per_frame = ds.PerFrameFunctionalGroupsSequence
    # The 4D segmentation data in ds.pixel_array: shape = (NumberOfFrames, rows, cols)
    seg_data_4d = ds.pixel_array

    # We'll store [z-position, slice_index] to help with constructing the affine.
    # Typically we rely on ImagePositionPatient in PlanePositionSequence to get
    # the (x,y,z) for each frame. Then we find the normal-axis coordinate
    # to sort slices or confirm ordering.
    positions = [None] * num_slices  # Will hold (index_in_vol, ImagePositionPatient) for each slice

    for frame_idx, frame_fg in enumerate(per_frame):
        # 4A) Identify the referenced SOPInstanceUID for this frame
        if hasattr(frame_fg, "DerivationImageSequence"):
            ref_uid = frame_fg.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
        elif hasattr(frame_fg, "ReferencedImageSequence"):
            ref_uid = frame_fg.ReferencedImageSequence[0].ReferencedSOPInstanceUID
        else:
            raise ValueError("Cannot find referenced SOPInstanceUID in frame functional groups.")

        # 4B) Find which slice index this UID corresponds to
        try:
            slice_idx = referenced_sop_uids.index(ref_uid)
        except ValueError:
            # Possibly the frame references an SOPInstanceUID not in ReferencedInstanceSequence.
            # Should be unusual, but handle gracefully.
            continue

        # 4C) Retrieve the segmentation mask for this frame
        frame_mask = seg_data_4d[frame_idx, :, :].astype(np.uint8)

        # 4D) Merge into seg_volume (if multiple segments overlap, we could do max, sum, or label)
        seg_volume[:, :, slice_idx] = np.maximum(seg_volume[:, :, slice_idx], frame_mask)

        # 4E) Store the slice's ImagePositionPatient if available
        plane_pos_seq = frame_fg.PlanePositionSequence[0]
        image_pos = np.array(plane_pos_seq.ImagePositionPatient, dtype=float)
        positions[slice_idx] = (slice_idx, image_pos)

    # ----------------------------------------------------------------
    # 5) Determine the final slice direction and origin for the LPS affine
    # ----------------------------------------------------------------
    # For a typical series, positions should be sorted along the cross(row_dir, col_dir) direction.
    # We'll check each stored position to see if we can detect a consistent spacing.
    slice_normal = np.cross(row_dir, col_dir)
    norm_len = np.linalg.norm(slice_normal)
    if norm_len > 0:
        slice_normal /= norm_len
    else:
        raise ValueError("Invalid orientation vectors: cross product is zero.")

    # Identify the first valid slice's position as the 'origin' in LPS
    # (lowest index that had a valid reference).
    valid_positions = [p for p in positions if p is not None]
    if len(valid_positions) == 0:
        raise ValueError("No valid positions found for any slice; cannot build geometry.")

    valid_positions.sort(key=lambda x: x[0])  # sort by slice_idx
    first_slice_idx, first_pos = valid_positions[0]

    # If slice_thickness wasn't set, attempt to guess from consecutive slices
    # that have valid positions.
    if not slice_thickness:
        slice_thickness = 1.0
        if len(valid_positions) > 1:
            second_slice_idx, second_pos = valid_positions[1]
            dist = np.dot((second_pos - first_pos), slice_normal)
            slice_thickness = abs(dist)

    # ----------------------------------------------------------------
    # 6) Create the LPS-to-voxel affine
    # ----------------------------------------------------------------
    # We scale row_dir by pixel_spacing[0], col_dir by pixel_spacing[1], slice_normal by slice_thickness.
    affine_lps = np.eye(4, dtype=float)
    affine_lps[0:3, 0] = row_dir * spacing_row
    affine_lps[0:3, 1] = col_dir * spacing_col
    affine_lps[0:3, 2] = slice_normal * slice_thickness
    affine_lps[0:3, 3] = first_pos

    # Create a NIfTI image in LPS orientation
    nifti_lps = nib.Nifti1Image(seg_volume, affine_lps)

    # ----------------------------------------------------------------
    # 7) Reorient to RAS to mimic dcm2niix
    # ----------------------------------------------------------------
    nifti_ras = nib.as_closest_canonical(nifti_lps)

    # ----------------------------------------------------------------
    # 8) Save to NIfTI
    # ----------------------------------------------------------------
    nib.save(nifti_ras, out_nifti_path)
    print(f"Saved 3D segmentation to {out_nifti_path} in RAS orientation.")

In [28]:
# Example usage
dicomseg_path = './in/SEG_20240507_121615_030_S6.dcm'
nifti_path = './seg.nii'
ref_nifti_path = '/Volumes/Siren/Brain_data/2.PatientDirectory/107/MRI/Processed_New/20120912_Mr_Brain_Wo+_W_Cont_Ss/Ax_T2_FLAIR.nii.gz'
dicomseg_to_nifti(dicomseg_path, nifti_path)

Saved 3D segmentation to ./seg.nii in RAS orientation.


In [41]:
import SimpleITK as sitk
sitk.ReadImage(dicomseg_path)
sitk.WriteImage(dicomseg_path, nifti_path)

TypeError: Wrong number or type of arguments for overloaded function 'ImageFileWriter_Execute'.
  Possible C/C++ prototypes are:
    itk::simple::ImageFileWriter::Execute(itk::simple::Image const &)
    itk::simple::ImageFileWriter::Execute(itk::simple::Image const &,itk::simple::PathType const &,bool,int)


In [6]:
""" A SimpleITK example demonstrating how to convert and resize DICOM files
    to common image types. """

import argparse
import csv
import functools
import itertools
import multiprocessing
import os
import sys

import SimpleITK as sitk


def convert_image(input_file_name, output_file_name, new_width=None):
    """ Convert a single DICOM image to a common image type. """
    try:
        image_file_reader = sitk.ImageFileReader()
        # only read DICOM images
        image_file_reader.SetImageIO("GDCMImageIO")
        image_file_reader.SetFileName(input_file_name)
        image_file_reader.ReadImageInformation()
        image_size = list(image_file_reader.GetSize())
        new_width = new_width if new_width else image_size[0]
        if len(image_size) == 3 and image_size[2] == 1:
            image_size[2] = 22
        image_file_reader.SetExtractSize(image_size)
        image = image_file_reader.Execute()
        if new_width:
            original_size = image.GetSize()
            original_spacing = image.GetSpacing()
            # new_spacing = [
            #     (original_size[0] - 1) * original_spacing[0] / (new_width - 1)
            # ] * 3
            new_spacing = original_spacing
            new_size = [
                new_width,
                image_size[1],
                22
            ]
            image = sitk.Resample(
                image1=image,
                size=new_size,
                transform=sitk.Transform(),
                interpolator=sitk.sitkLinear,
                outputOrigin=image.GetOrigin(),
                outputSpacing=new_spacing,
                outputDirection=image.GetDirection(),
                defaultPixelValue=0,
                outputPixelType=image.GetPixelID(),
            )
        # If a single channel image, rescale to [0,255]. Also modify the
        # intensity values based on the photometric interpretation. If
        # MONOCHROME2 (minimum should be displayed as black) we don't need to
        # do anything, if image has MONOCRHOME1 (minimum should be displayed as
        # white) we flip # the intensities. This is a constraint imposed by ITK
        # which always assumes MONOCHROME2.
        if image.GetNumberOfComponentsPerPixel() == 1:
            image = sitk.RescaleIntensity(image, 0, 255)
            if image_file_reader.GetMetaData("0028|0004").strip() == "MONOCHROME1":
                image = sitk.InvertIntensity(image, maximum=255)
            image = sitk.Cast(image, sitk.sitkUInt8)
        sitk.WriteImage(image, output_file_name)
        return True
    except RuntimeError as e:
        print(e)
        return False


def convert_images(input_file_names, output_file_names, new_width):
    """ Convert multiple DICOM images in parallel to a common image type. """
    MAX_PROCESSES = 15
    with multiprocessing.Pool(processes=MAX_PROCESSES) as pool:
        return pool.starmap(
            functools.partial(convert_image, new_width=new_width),
            zip(input_file_names, output_file_names),
        )


def positive_int(int_str):
    """ Custom argparse type for positive integers. """
    value = int(int_str)
    if value <= 0:
        raise argparse.ArgumentTypeError(int_str + " is not a positive integer value")
    return value


def directory(dir_name):
    """ Custom argparse type for directory. """
    if not os.path.isdir(dir_name):
        raise argparse.ArgumentTypeError(dir_name + " is not a valid directory name")
    return dir_name


def main(argv=None):
    """ Main function. """
    parser = argparse.ArgumentParser(
        description="Convert and resize DICOM files to common image types."
    )
    parser.add_argument(
        "root_of_data_directory",
        type=directory,
        help="Path to the topmost directory containing data.",
    )
    parser.add_argument(
        "output_file_extension",
        help="Image file extension, this determines output file type " "(e.g. png) .",
    )
    parser.add_argument("--w", type=positive_int, help="Width of converted images.")
    parser.add_argument("--od", type=directory, help="Output directory.")
    args = parser.parse_args(argv)

    input_file_names = []
    for dir_name, _, file_names in os.walk(args.root_of_data_directory):
        input_file_names += [
            os.path.join(os.path.abspath(dir_name), fname) for fname in file_names
        ]
    if args.od:
        # if all output files are written to the same directory we need them
        # to have a unique name, so use an index.
        file_names = [
            os.path.join(os.path.abspath(args.od), str(i))
            for i in range(len(input_file_names))
        ]
    else:
        file_names = input_file_names
    output_file_names = [
        file_name + "." + args.output_file_extension for file_name in file_names
    ]

    res = convert_images(input_file_names, output_file_names, args.w)
    input_file_names = list(itertools.compress(input_file_names, res))
    output_file_names = list(itertools.compress(output_file_names, res))

    # save csv file mapping input and output file names.
    # using csv module and not pandas so as not to create more dependencies
    # for the examples. pandas based code is more elegant/shorter.
    dir_name = args.od if args.od else os.getcwd()
    with open(os.path.join(dir_name, "file_names.csv"), mode="w", encoding='utf-8') as fp:
        fp_writer = csv.writer(
            fp, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        fp_writer.writerow(["input file name", "output file name"])
        for data in zip(input_file_names, output_file_names):
            fp_writer.writerow(data)



convert_image("./in/SEG_20240507_121615_030_S6.dcm", './seg.nii', new_width=None)

True

In [4]:
import sys
import SimpleITK as sitk

def main(args):
    """ A SimpleITK script that prints a DICOM image's tags. """
    if len(args) < 2:
        print("Usage: DicomImagePrintTags <input_file>")
        sys.exit(1)

    reader = sitk.ImageFileReader()

    reader.SetFileName(args[1])
    reader.LoadPrivateTagsOn()

    reader.ReadImageInformation()
    reader.

    for k in reader.GetMetaDataKeys():
        v = reader.GetMetaData(k)
        print(f'({k}) = = "{v}"')

    print(f"Image Size: {reader.GetSize()}")
    print(f"Image PixelType: {sitk.GetPixelIDValueAsString(reader.GetPixelID())}")

main(['bs','./in/SEG_20240507_121615_030_S6.dcm'])

(0008|0008) = = "DERIVED\PRIMARY "
(0008|0016) = = "1.2.840.10008.5.1.4.1.1.66.4"
(0008|0018) = = "2.25.945894488964878615568549078137573714889"
(0008|0020) = = "20120920"
(0008|0021) = = "20240507"
(0008|0023) = = "20240507"
(0008|0030) = = ""
(0008|0031) = = "121620"
(0008|0033) = = "121620"
(0008|0050) = = ""
(0008|0060) = = "SEG "
(0008|0070) = = "ICR "
(0008|0090) = = ""
(0008|103e) = = "FH_ET "
(0008|1090) = = "XNAT-OHIF-VIEWER"
(0010|0010) = = ""
(0010|0020) = = ""
(0010|0030) = = ""
(0010|0040) = = ""
(0010|1010) = = ""
(0018|1000) = = "1 "
(0018|1020) = = "3.4.1 "
(0018|9004) = = "RESEARCH"
(0020|000d) = = "2.25.159702042684316084112504011909566489793"
(0020|000e) = = "2.25.320496192564362276577358453783751916420"
(0020|0010) = = "2.25.302288137327796363342940126751124909674"
(0020|0011) = = "99"
(0020|0013) = = "1 "
(0020|0052) = = "2.25.95477689593731099410504450315468431284"
(0020|1040) = = ""
(0020|4000) = = "NOT FOR CLINICAL USE"
(0028|0002) = = "1"
(0028|0004) = = "MONOC

In [27]:
import nibabel as nib

img = nib.load("AXIAL_T2_FLAIR.nii.gz")
mask = nib.load("seg.nii")
nii = nib.load("in/in_FH_ET_20120920000000_99.nii.gz")

print("Image Metadata:")
print(img.affine)
print("Mask Metadata:")
print(mask.affine)
print("NII Metadata:")
print(nii.affine)

Image Metadata:
[[-4.29006040e-01 -2.30409000e-02  1.21999599e-01  1.16505997e+02]
 [ 2.22935230e-02 -4.27600890e-01 -5.45998216e-01  1.26185997e+02]
 [ 9.94029269e-03 -3.56170125e-02  6.47587872e+00 -6.69872971e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Mask Metadata:
[[-9.98385906e-01 -5.36209121e-02  1.87197290e-02  1.17844002e+02]
 [ 5.18703535e-02 -9.95115280e-01 -8.39947015e-02  1.20181000e+02]
 [ 2.31321603e-02 -8.28881189e-02  9.96290326e-01  4.24744987e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
NII Metadata:
[[-4.29006130e-01  2.30409056e-02  1.21678233e-01  1.05340096e+02]
 [ 2.22935714e-02  4.27601010e-01 -5.45965493e-01 -9.50481186e+01]
 [ 9.94029455e-03  3.56170237e-02  6.47588634e+00 -5.28081970e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [7]:
import SimpleITK as sitk

# Read image and mask
image = sitk.ReadImage("AXIAL_T2_FLAIR.nii.gz")
mask = sitk.ReadImage("seg.nii")

# Resample the mask to match the image
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(image)
resampler.SetInterpolator(sitk.sitkNearestNeighbor)  # Use nearest neighbor for masks
resampled_mask = resampler.Execute(mask)

# Save the aligned mask
sitk.WriteImage(resampled_mask, "aligned_mask.nii.gz")

In [ ]:
import numpy as np
def create_affine(ipp, iop, ps):
    """Generate an affine matrix from DICOM IPP and IOP attributes.
    The ipp (ImagePositionPatient) parameter should an Nx3 array, and
    the iop (ImageOrientationPatient) parameter should be Nx6, where
    N is the number of DICOM slices in the series.
    The return values are the affine matrix and the pixdim.
    Note the the output will use DICOM anatomical coordinates:
    x increases towards the left, y increases towards the back.
    """
    # solve Ax = b where x is slope, intecept
    n = ipp.shape[0]
    A = np.column_stack([np.arange(n), np.ones(n)])
    x, r, rank, s = np.linalg.lstsq(A, ipp, rcond=None)
    # round small values to zero
    x[(np.abs(x) < 1e-6)] = 0.0
    vec = x[0,:] # slope
    pos = x[1,:] # intercept

    # pixel spacing should be the same for all image
    spacing = np.ones(3)
    spacing[1::-1] = ps[0,:]
    if np.sum(np.abs(ps - spacing[1::-1])) > spacing[0]*1e-6:
        sys.stderr.write("Pixel spacing is inconsistent!\n");

    # compute slice spacing, round to 7 decimal places
    spacing[2] = np.round(np.sqrt(np.sum(np.square(vec))), 7)

    # get the orientation
    iop_average = np.mean(iop, axis=0)
    u = iop_average[0:3]
    u /= np.sqrt(np.sum(np.square(u)))
    v = iop_average[3:6]
    v /= np.sqrt(np.sum(np.square(v)))

    # round small values to zero
    u[(np.abs(u) < 1e-6)] = 0.0
    v[(np.abs(v) < 1e-6)] = 0.0

    # create the matrix
    mat = np.eye(4)
    mat[0:3,0] = u*spacing[0]
    mat[0:3,1] = v*spacing[1]
    mat[0:3,2] = vec
    mat[0:3,3] = pos

    # check whether slice vec is orthogonal to iop vectors
    dv = np.dot(vec, np.cross(u, v))
    qfac = np.sign(dv)
    if np.abs(qfac*dv - spacing[2]) > 1e-6:
        sys.stderr.write("Non-orthogonal volume!\n");

    # compute the pixdim array, with qfac as the first element
    pixdim = np.hstack([np.array(qfac), spacing])

    return mat, pixdim

import pydicom
ds = pydicom.dcmread("in/SEG_20240507_121548_916_S9.dcm")
create_affine(np.array(ds.PerFrameFunctionalGroupsSequence[0].PlanePositionSequence[0].ImagePositionPatient), ds.SharedFunctionalGroupsSequence[0].PlaneOrientationSequence[0].ImageOrientationPatient, 22)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [2]:
import numpy as np
import highdicom
from dcmseg2nii import format_nifti, write_nifti
import SimpleITK as sitk
nii = sitk.ReadImage('/Volumes/Siren/Brain_data/2.PatientDirectory/107/MRI/Processed_New/20120912_Mr_Brain_Wo+_W_Cont_Ss/T1C.nii.gz')
ds = highdicom.seg.segread("in/SEG_20240507_121548_916_S9.dcm")
uids = [x[2] for x in ds.get_source_image_uids()]
pixels = ds.get_pixels_by_source_instance(uids, segment_numbers=[1], ignore_spatial_locations=True, combine_segments=True)
# pixels = pixels.transpose(1,2,0)
# print(pixels.shape)
# nifti = format_nifti(pixels, nii)
# write_nifti(nifti, "out.nii.gz")
for slice in pixels:
    print(slice.shape, np.unique(slice))

(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0 1]
(512, 512) [0 1]
(512, 512) [0 1]
(512, 512) [0]
(512, 512) [0 1]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]
(512, 512) [0]


In [22]:
ds.SharedFunctionalGroupsSequence[0].PlaneOrientationSequence[0].ImageOrientationPatient
ds.PerFrameFunctionalGroupsSequence[0].PlanePositionSequence[0].ImagePositionPatient

[-118.453, -117.451, 36.6269]

In [3]:
import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetFileName("AXIAL_T2_FLAIR.nii.gz")
x: sitk.Image = reader.Execute()
nii_out = sitk.Cast(x, sitk.sitkUInt8)
x is nii_out

False

In [7]:
x.Copy

('ITK_FileNotes',
 'ITK_original_direction',
 'ITK_original_spacing',
 'ITK_sform_corrected',
 'aux_file',
 'bitpix',
 'cal_max',
 'cal_min',
 'datatype',
 'descrip',
 'dim[0]',
 'dim[1]',
 'dim[2]',
 'dim[3]',
 'dim[4]',
 'dim[5]',
 'dim[6]',
 'dim[7]',
 'dim_info',
 'intent_code',
 'intent_name',
 'intent_p1',
 'intent_p2',
 'intent_p3',
 'nifti_type',
 'pixdim[0]',
 'pixdim[1]',
 'pixdim[2]',
 'pixdim[3]',
 'pixdim[4]',
 'pixdim[5]',
 'pixdim[6]',
 'pixdim[7]',
 'qfac',
 'qform_code',
 'qform_code_name',
 'qoffset_x',
 'qoffset_y',
 'qoffset_z',
 'qto_xyz',
 'quatern_b',
 'quatern_c',
 'quatern_d',
 'scl_inter',
 'scl_slope',
 'sform_code',
 'sform_code_name',
 'slice_code',
 'slice_duration',
 'slice_end',
 'slice_start',
 'srow_x',
 'srow_y',
 'srow_z',
 'toffset',
 'vox_offset',
 'xyzt_units')

In [ ]:
from pathlib import Path

import highdicom as hd
import numpy as np
from pydicom.sr.codedict import codes
from pydicom.filereader import dcmread

# Path to directory containing single-frame legacy CT Image instances
# stored as PS3.10 files
series_dir = Path('path/to/series/directory')
image_files = series_dir.glob('*.dcm')

# Read CT Image data sets from PS3.10 files on disk
image_datasets = [dcmread(str(f)) for f in image_files]

# Create a binary segmentation mask
mask = np.zeros(
    shape=(
        len(image_datasets),
        image_datasets[0].Rows,
        image_datasets[0].Columns
    ),
    dtype=np.bool
)
mask[1:-1, 10:-10, 100:-100] = True

# Describe the algorithm that created the segmentation
algorithm_identification = hd.AlgorithmIdentificationSequence(
    name='test',
    version='v1.0',
    family=codes.cid7162.ArtificialIntelligence
)

# Describe the segment
description_segment_1 = hd.seg.SegmentDescription(
    segment_number=1,
    segment_label='first segment',
    segmented_property_category=codes.cid7150.Tissue,
    segmented_property_type=codes.cid7166.ConnectiveTissue,
    algorithm_type=hd.seg.SegmentAlgorithmTypeValues.AUTOMATIC,
    algorithm_identification=algorithm_identification,
    tracking_uid=hd.UID(),
    tracking_id='test segmentation of computed tomography image'
)

# Create the Segmentation instance
seg_dataset = hd.seg.Segmentation(
    source_images=image_datasets,
    pixel_array=mask,
    segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
    segment_descriptions=[description_segment_1],
    series_instance_uid=hd.UID(),
    series_number=2,
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer='Manufacturer',
    manufacturer_model_name='Model',
    software_versions='v1',
    device_serial_number='Device XYZ',
).DimensionIndexSequence

print(seg_dataset)

seg_dataset.save_as("seg.dcm")

In [17]:
import nibabel as nib
nii = nib.load("/Volumes/Siren/Brain_data/1.PatientDirectory/158/MRI/Processed_New/20181030_Mr_Brain__Fh_Non_Org/3D_Flair_FS_11003_3D_Flair_FS_20181030131442_11003.nii.gz")
# nii = nib.load("/Volumes/Siren/Brain_data/1.PatientDirectory/150/MRI/Processed_New/20180328_MR_Brain/AX_FLAIR_AX_FLAIR_20180328084507_9a.nii.gz")
# nii = nib.load("/Volumes/Siren/Brain_data/1.PatientDirectory/158/MRI/Processed_New/20181030_Mr_Brain__Fh_Non_Org/FLAIR.nii.gz")
nib.aff2axcodes(nii.affine)

('R', 'A', 'S')

In [19]:

nii = nib.load("/Volumes/Siren/Brain_data/1.PatientDirectory/158/MRI/Processed_New/20181030_Mr_Brain__Fh_Non_Org/FLAIR.nii.gz")
nii.affine
nib.aff2axcodes(nii.affine)

('L', 'P', 'S')

In [21]:
nii = nib.load("/Volumes/Siren/Brain_data/1.PatientDirectory/158/MRI/Processed_New/20181030_Mr_Brain__Fh_Non_Org/xnat/3D_Flair_FS_11003_3D_Flair_FS_20181030131442_11003_Edema.nii.gz")
nii.affine
nib.aff2axcodes(nii.affine)

('R', 'A', 'S')

In [22]:
nii.get_fdata().shape

(332, 512, 52)

In [9]:
import SimpleITK as sitk
import highdicom as hd
import nibabel as nib
from dcmseg2nii import get_affine_from_sitk
# nib.aff2axcodes(get_affine_from_sitk(sitk.GetImageFromArray(hd.seg.segread('in/SEG_20240507_121615_030_S6.dcm'))))
sitk.ReadImage('in/SEG_20240507_121548_916_S9.dcm').GetSize()

(512, 512, 8)